In [2]:
# Muhammad Ali Khan
# Section-B
# 21i-0429
# AI-Project



import numpy as np
from random import random
import csv


# Defining constants
No_of_Days = 5
No_of_Slots_each_day = 6
No_of_Courses_per_week = 2
No_of_Lab_Courses_per_week = 1
No_of_rooms = 10
No_of_profs = 25
No_of_sections = 5

# Defining meaningful names
SECTIONS = ['Section A', 'Section B', 'Section C', 'Section D', 'Section E']
professors = [
    'Professor Harry', 'Professor Zaid', 'Professor John', 'Professor Ayra', 'Professor Sarah',
    'Professor Mathew', 'Professor William', 'Professor Camilla', 'Professor Imran', 'Professor Shams',
    'Professor Alice', 'Professor Benjamin', 'Professor Clara', 'Professor David', 'Professor Emma',
    'Professor Felix', 'Professor Grace', 'Professor Henry', 'Professor Isabella', 'Professor Jack',
    'Professor Katherine', 'Professor Liam', 'Professor Maria', 'Professor Nathan', 'Professor Olivia'
]
timeSlot_each_day = ["8:30 - 9:50", "10:05 - 11:25", "11:40 - 1:00", "1:15 - 2:35", "2:50 - 4:10", "4:25 - 5:45"]
name_of_day = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
SECTION_STRENGTHS = {'Section A': 55, 'Section B': 57, 'Section C': 44, 'Section D': 67, 'Section E': 39}


# Defining course types
THEORY = 0
LAB = 1

# Defining room types and names
class_name = [f"CR {i+1}" for i in range(6)]  # Capacity 60
large_hall_name = [f"LH {i+1}" for i in range(4)]  # Capacity 120
all_rooms = class_name + large_hall_name
max_capacity = [60] * 6 + [120] * 4

# Defining courses (unique for each section)
COURSES = {
    'Section A': ['Object Oriented Programming', 'Digital Logic Design', 'Differential Equations', 'Pakistan Studies','Object Oriented Programming Lab'],
    'Section B': ['Data Structures', 'Discrete Structures', 'Computer Organization and Assembly Language', 'Linear Algebra', 'Data Structures Lab'],
    'Section C': ['Database Systems', 'Operating Systems', 'Probability and Statistics', 'Design and Analysis of Algorithms', 'Database Systems Lab'],
    'Section D': ['Computer Networks', 'Software Design and Analysis', 'Theory of Automata', 'Statistical Modelling', 'Computer Networks Lab'],
    'Section E': ['Artificial Intelligence', 'Numerical Computing', 'Software Engineering', 'Software for Mobile Devices', 'Artificial Intelligence Lab']
}

# Professor and section limits
Max_courses_per_prof = 3
Max_courses_per_section = 5

# Break length
BREAK_LENGTH = 15

NUM_BITS_PER_SLOT = 1 + 3 + 3 + 3 + 3 + 4

# Global variable to keep track of labs scheduled for each section
section_labs = {section: 0 for section in SECTIONS}


# Function to initialize the population with random bitstrings
def initialize_population(n_pop, n_bits):
    return [np.random.randint(0, 2, n_bits).tolist() for _ in range(n_pop)]

# Tournament selection
def selection(pop, scores, k=3):
    selection_ix = np.random.randint(len(pop))
    for ix in np.random.randint(0, len(pop), k-1):
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

# Crossover operator
def crossover(p1, p2, r_cross):
    c1, c2 = p1.copy(), p2.copy()
    if random() < r_cross:
        pt = np.random.randint(1, len(p1)-2)
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]

# Mutation operator
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        if random() < r_mut:
            bitstring[i] = 1 - bitstring[i]

def fitness_function(candidate):
    decoded_candidate = decode_candidate(candidate)
    conflicts = calculate_conflicts(decoded_candidate)

    # Penalty for multiple labs per section
    for section in SECTIONS:
        lab_count = sum(slot["course_type"] == "LAB" for day_schedule in decoded_candidate for slot in day_schedule if slot["section"] == section)
        if lab_count > 1:
            conflicts.append(lab_count - 1)  # Penalize each extra lab

    # Penalty for more than 2 classes of the same course in a week
    course_counts = {course: 0 for courses in COURSES.values() for course in courses}
    for section_schedule in decoded_candidate:
        for slot in section_schedule:
            course_counts[slot["course_name"]] += 1
            if course_counts[slot["course_name"]] > 2:
                conflicts.append(1)  # Penalize if more than 2 classes of the same course in a week

    return -sum(conflicts)



# Decode candidate
def decode_candidate(candidate):
    timetable = []
    for day in range(No_of_Days):
        daily_schedule = []
        for slot in range(No_of_Slots_each_day):
            start_index = (day * No_of_Slots_each_day + slot) * NUM_BITS_PER_SLOT
            end_index = start_index + NUM_BITS_PER_SLOT
            slot_info = decode_slot(candidate[start_index:end_index])
            daily_schedule.append(slot_info)
        timetable.append(daily_schedule)
    return timetable



def decode_slot(slot):
    course_type = "THEORY" if slot[0] == 0 else "LAB"
    section_index = int("".join(str(x) for x in slot[1:4]), 2) % No_of_sections
    section = SECTIONS[section_index]
    professor_index = int("".join(str(x) for x in slot[4:7]), 2) % No_of_profs
    professor = professors[professor_index]  # Fix here
    day_index = int("".join(str(x) for x in slot[7:10]), 2) % No_of_Days
    lecture_day = name_of_day[day_index]
    time_slot_index = int("".join(str(x) for x in slot[10:13]), 2) % No_of_Slots_each_day
    lecture_timeslot = timeSlot_each_day[time_slot_index]

    if course_type == "LAB":
        global section_labs
        if section_labs[section] >= No_of_Lab_Courses_per_week:
            # This section already has a lab, so schedule a theory course instead
            course_name = COURSES[section][time_slot_index % (len(COURSES[section]) - 1)]
        else:
            lab_course = None
            if section == 'Section A':
                lab_course = 'Object Oriented Programming Lab'
            elif section == 'Section B':
                lab_course = 'Data Structures Lab'
            elif section == 'Section C':
                lab_course = 'Database Systems Lab'
            elif section == 'Section D':
                lab_course = 'Computer Networks Lab'
            elif section == 'Section E':
                lab_course = 'Artificial Intelligence Lab'
            course_name = lab_course
            section_labs[section] += 1
    else:
        course_name = COURSES[section][time_slot_index % len(COURSES[section])]

    # Assign room based on section
    if section == 'Section D':
        # Assign LH room for Section D
        lh_rooms = [room for room in all_rooms if room.startswith("LH")]
        lecture_room = np.random.choice(lh_rooms)
    else:
        # Assign CR room for other sections
        cr_rooms = [room for room in all_rooms if room.startswith("CR")]
        lecture_room = np.random.choice(cr_rooms)

    return {
        "course_type": course_type,
        "course_name": course_name,
        "section": section,
        "professor": professor,
        "lecture_day": lecture_day,
        "lecture_timeslot": lecture_timeslot,
        "lecture_room": lecture_room
    }



# Calculate conflicts
def calculate_conflicts(timetable):
    conflicts = []

    # 1. Professor conflicts (teaching two courses at the same time)
    for day_schedule in timetable:
        assigned_professors = set()
        for slot in day_schedule:
            if slot["course_type"] == "THEORY":
                if slot["professor"] in assigned_professors:
                    conflicts.append(1)  # Professor conflict
                else:
                    assigned_professors.add(slot["professor"])

    # 2. Section conflicts (assigned to two rooms at the same time)
    for day_schedule in timetable:
        assigned_sections = set()
        for slot in day_schedule:
            if slot["section"] in assigned_sections:
                conflicts.append(1)  # Section conflict
            else:
                assigned_sections.add(slot["section"])

    # 3. Room conflicts (assigned to two sections at the same time) and Room capacity
    for day_schedule in timetable:
        assigned_rooms = set()
        for slot in day_schedule:
            if slot["lecture_room"] in assigned_rooms:
                conflicts.append(1)  # Room conflict
            else:
                assigned_rooms.add(slot["lecture_room"])
            # Room capacity check
            room_index = all_rooms.index(slot["lecture_room"])
            if max_capacity[room_index] < 60:
                conflicts.append(1)

    # 4. Room capacity constraints
    for day_schedule in timetable:
        for slot in day_schedule:
            room_index = all_rooms.index(slot["lecture_room"])
            if max_capacity[room_index] < 60:  # Assuming all sections have at least 60 students
                conflicts.append(1)

    # 5. Professor workload constraint
    professor_counts = {}
    for professor in professors:  # Fix here
        professor_counts[professor] = 0
    for day_schedule in timetable:
        for slot in day_schedule:
            professor_counts[slot["professor"]] += 1
    for count in professor_counts.values():
        if count > Max_courses_per_prof:
            conflicts.append(1)

    # 6. Section workload constraint
    section_counts = {}
    for section in SECTIONS:
        section_counts[section] = 0
    for day_schedule in timetable:
        for slot in day_schedule:
            section_counts[slot["section"]] += 1
    for count in section_counts.values():
        if count > Max_courses_per_section:
            conflicts.append(1)

    # 7. Course scheduling constraints (two lectures per week, not on same/adjacent days)
    for section in SECTIONS:
        for course_name in COURSES[section]:
            lecture_days = []
            for day_schedule in timetable:
                for slot in day_schedule:
                    if slot["section"] == section and slot["course_name"] == course_name:
                        lecture_days.append(day_schedule.index(slot))
            if len(lecture_days) != No_of_Courses_per_week:
                conflicts.append(1)
            elif abs(lecture_days[0] - lecture_days[1]) <= 1:
                conflicts.append(1)

    # 8. Lab constraints (two consecutive slots)
    for day_schedule in timetable:
        for i, slot in enumerate(day_schedule[:-1]):
            if slot["course_type"] == "LAB" and day_schedule[i+1]["course_type"] != "LAB":
                conflicts.append(1)

    # 9. Break constraints (15 mins between classes)
    for day_index in range(len(timetable)-1):
        day_schedule = timetable[day_index]
        for slot_index in range(len(day_schedule)-1):
            slot = day_schedule[slot_index]
            if slot["lecture_timeslot"] != "5:15 - 6:45":
                next_slot = timetable[day_index][slot_index + 1]
                next_slot_start = int(next_slot["lecture_timeslot"].split(" - ")[0].split(":")[0])
                current_slot_end = int(slot["lecture_timeslot"].split(" - ")[1].split(":")[0])
                if next_slot_start - current_slot_end < (BREAK_LENGTH / 60):
                    conflicts.append(1)

     # 10. More than 2 classes of the same theory course in a week for each section
    course_counts = {section: {course: 0 for course in COURSES[section]} for section in SECTIONS}
    for section_schedule in timetable:
        for slot in section_schedule:
            if slot["course_type"] == "THEORY":
                course_counts[slot["section"]][slot["course_name"]] += 1

    for section, courses in course_counts.items():
        for course, count in courses.items():
            if count > 2:
                conflicts.append(1)






    # Soft Constraints


# 1. Theory classes in the morning, labs in the afternoon
    for day_schedule in timetable:
        for slot in day_schedule:
            if slot["course_type"] == "THEORY" and slot["lecture_timeslot"].split(" - ")[0] in ["8:30", "10:05", "11:40"]:
                conflicts.append(1)  # Theory class in the afternoon
            elif slot["course_type"] == "LAB" and slot["lecture_timeslot"].split(" - ")[0] in ["1:15", "2:50", "4:25"]:
                conflicts.append(1)  # Lab session in the morning

    # 2. Minimize traversing floors
    for day_schedule in timetable:
        for i in range(len(day_schedule) - 1):
            current_slot = day_schedule[i]
            next_slot = day_schedule[i + 1]
            if current_slot["lecture_room"][:2] != next_slot["lecture_room"][:2]:
                conflicts.append(1)  # Rooms on different floors

    # 3. Classes in the same classroom throughout the week
    course_rooms = {section: {} for section in SECTIONS}
    for day_schedule in timetable:
        for slot in day_schedule:
            if slot["course_name"] in course_rooms[slot["section"]]:
                if course_rooms[slot["section"]][slot["course_name"]] != slot["lecture_room"]:
                    conflicts.append(1)  # Course held in different classrooms on different days
            else:
                course_rooms[slot["section"]][slot["course_name"]] = slot["lecture_room"]

    # 4. Prefer longer blocks of continuous teaching time
    for day_schedule in timetable:
        same_teacher_slots = {}
        for slot in day_schedule:
            if slot["professor"] in same_teacher_slots:
                if same_teacher_slots[slot["professor"]]["course_name"] != slot["course_name"]:
                    if int(slot["lecture_timeslot"].split(" - ")[0].split(":")[0]) != int(same_teacher_slots[slot["professor"]]["lecture_timeslot"].split(" - ")[1].split(":")[0]) + 1:
                        conflicts.append(1)  # Non-consecutive blocks for the same teacher
            same_teacher_slots[slot["professor"]] = slot

    return conflicts



# Genetic algorithm function
def genetic_algorithm(fitness_fn, n_bits, n_gen, n_pop, r_cross, r_mut):
    pop = initialize_population(n_pop, n_bits)
    best, best_eval = 0, fitness_fn(pop[0])
    for gen in range(n_gen):
        scores = [fitness_fn(c) for c in pop]
        for i in range(n_pop):
            if scores[i] > best_eval:
                best, best_eval = pop[i], scores[i]
                print(f">{gen}, new best f({best}) = {best_eval}")
        selected = [selection(pop, scores) for _ in range(n_pop)]
        children = list()
        for i in range(0, n_pop, 2):
            p1, p2 = selected[i], selected[i+1]
            for c in crossover(p1, p2, r_cross):
                mutation(c, r_mut)
                children.append(c)
        pop = children
    return [best, best_eval]

# Example usage
n_bits = No_of_Days * No_of_Slots_each_day * NUM_BITS_PER_SLOT
best_solution, best_fitness = genetic_algorithm(fitness_function, n_bits, 100, 100, 0.9, 1.0/float(n_bits))

# Output the timetable
decoded_best_solution = decode_candidate(best_solution)
for i, day_schedule in enumerate(decoded_best_solution):
    print(f"\n=== {name_of_day[i]} ===")
    # Sort the day_schedule based on lecture_timeslot
    day_schedule.sort(key=lambda x: timeSlot_each_day.index(x['lecture_timeslot']))
    for slot in day_schedule:
        print(f"{slot['lecture_timeslot']}: - {slot['course_name']} - {slot['section']} with {slot['professor']} in {slot['lecture_room']}")

# Display the best fitness
print("\nBest Fitness:", best_fitness)




# Function to write the timetable to a CSV file
def write_to_csv(timetable, filename):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Day', 'Time Slot', 'Course Name', 'Section', 'Professor', 'Room'])

        for i, day_schedule in enumerate(timetable):
            for slot in day_schedule:
                writer.writerow([name_of_day[i], slot['lecture_timeslot'], slot['course_name'], slot['section'], slot['professor'], slot['lecture_room']])

# Example usage
write_to_csv(decoded_best_solution, 'timetable.csv')


>0, new best f([1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 